In [1]:
"""Package Loading"""
"""General"""
from webscraping_dictionaries import *
from webscraping_functions import *
import time
from dateutil import relativedelta
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By  
import requests
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import ssl
import os
#import dotenv
"""Chrome Options"""
#from selenium.webdriver import Chrome 
#from selenium.webdriver.chrome.service import Service 
#from webdriver_manager.chrome import ChromeDriverManager

"""Firefox Options"""
#from selenium.webdriver import Firefox
#from selenium.webdriver.firefox.service import Service
#from webdriver_manager.firefox import GeckoDriverManager


"""Initialize the driver"""

"""Driver Setup"""

#following steps from here: https://www.zenrows.com/blog/scraping-javascript-rendered-web-pages#installing-the-requirements
#critical help using the click method from here https://www.geeksforgeeks.org/click-element-method-selenium-python/

"""Chrome Version"""
#options = webdriver.ChromeOptions()
#options.headless = True
#options.page_load_strategy = 'none' 
#chrome_path = ChromeDriverManager().install()
#chrome_service = Service(chrome_path)
#driver = Chrome(options=options, service=chrome_service) 
#driver.implicitly_wait(5)

"""Firefox Version"""  
#options = webdriver.FirefoxOptions()
#options.headless = True
#options.add_argument("--headless=new")
#options.page_load_strategy = 'none' 
#firefox_path = GeckoDriverManager().install() 
#firefox_service = Service(firefox_path)
#driver = Firefox(options=options, service=firefox_service)
#driver.implicitly_wait(5)

'Firefox Version'

"""DEBUGGING SANDBOX"""

In [ ]:
#table this for now, since we still want to check for comp plan and ordinance keywords anyway
"Charlotte County Solar Development Page"
def charlotte_county_solar_development():
    driver.get("https://www.charlottecountyva.gov/departments/planning___zoning/solar_development.php")
    time.sleep(1)
    table_rows = driver.find_elements(By.CSS_SELECTOR,"tr")
    

In [ ]:
#come back to this another day
all_dictionaries=[agendacenter_dictionary,boarddocs_dictionary,civicclerk_dictionary,civicweb_dictionary,document_center_dictionary,escribe_dictionary,event_list_dictionary,granicus_dictionary,granicus_2_dictionary,laserfiche_dictionary,legistar_dictionary,meetingstable_dictionary,novusagenda_dictionary,php_table_dictionary,primegov_dictionary,county_dictionary_single_variable,county_dictionary_double_variable,city_dictionary_single_variable,city_dictionary_double_variable]

for dictionary in all_dictionaries:
    for entry in dictionary:
        print(entry)

In [15]:
#can I standardize the PDF search?
def search_pdf(meeting_links_list, locality):
    messages=[]
    for item in meeting_links_list:
        driver.get(item)
        time.sleep(10)
        pages = driver.find_elements(By.CSS_SELECTOR,"div[class*=page")
        for page in pages:
            driver.execute_script("arguments[0].scrollIntoView();", page)
            time.sleep(2)
            agenda_content = driver.find_elements(By.CSS_SELECTOR,'div[class*=textLayer')
            readable = check_readability(agenda_content)
            if readable==True:
                agenda_search = search_agenda_for_keywords(agenda_content)
                if agenda_search != []:
                    messages.append("Keyword(s) " + ", ".join(agenda_search) + " found in upcoming meeting for " + locality + ". " + item)
            elif readable==False:
                messages.append("New meeting document available for " + locality + ". Part of the document cannot be scanned for keywords. " + item)
    return pd.Series(messages).unique().tolist()

In [3]:
def virginia_beach_cc(url):
    driver.get(url)
    time.sleep(10)
    messages = []
    current_agenda = driver.find_element(By.CSS_SELECTOR,"a[href*=CurrentBriefAgenda")
    agenda_url = current_agenda.get_attribute("href")
    driver.get(agenda_url)
    time.sleep(10)
    agenda_content = driver.find_elements(By.CSS_SELECTOR,"div[class*=textLayer")
    agenda_search = search_agenda_for_keywords(agenda_content)
    if agenda_search != []:
        messages.append("Keyword(s) " + ", ".join(agenda_search) + " found in upcoming meeting for Virginia Beach in latest City Council meeting agenda. " + agenda_url)
    return messages

In [4]:
virginia_beach_cc(city_dictionary_single_variable['Virginia Beach City Council'][1])

[]

In [16]:
"""Sussex County"""
def sussex_county(url):
    driver.get(url)
    time.sleep(10)
    list_rows = driver.find_elements(By.CSS_SELECTOR,"li")
    meetings = [item for item in list_rows if "Agenda" in item.text]
    current_meetings = [item for item in meetings if "Archived" not in item.text]
    future_meetings = [item for item in current_meetings if check_meeting_date(item.text)==True]
    document_links =[]
    for item in future_meetings:
        documents = item.find_elements(By.CSS_SELECTOR,"a")
        for item in documents:
            document_links.append(item.get_attribute("href")) 
    messages = search_pdf(document_links,"Sussex County")
    return messages

In [3]:
driver.get("https://www.sussexcountyva.gov/page/agendas-and-minutes/")
time.sleep(10)

In [4]:
list_rows = driver.find_elements(By.CSS_SELECTOR,"li")

In [5]:
meetings = [item for item in list_rows if "Agenda" in item.text]

In [6]:
current_meetings = [item for item in meetings if "Archived" not in item.text]

In [7]:
future_meetings = [item for item in current_meetings if check_meeting_date(item.text)==True]

In [17]:
sussex_county("https://www.sussexcountyva.gov/page/agendas-and-minutes/")

KeyboardInterrupt: 